In [ ]:
# =============================================================================
# 小売業需要予測モデル - マスタテーブル方式改善記録
# =============================================================================
# 
# 【実施した施策】
# - n_estimatorsを100に戻してベースラインに設定
# - データクレンジング：欠損値と外れ値の除去
# - マスタテーブル方式：商品・店舗マスタを事前作成
# - 利益重視特徴量：限界利益20%を仮定した商品分類
# - 店舗補正：店舗ごとの売上特性を考慮した特徴量
# 
# 【スコア】
# - ベースライン（線形回帰）: 3.9937572546850784
# - 前回の結果（rf100）: 3.07256739424164
# - 今回の結果: 3.644648  ← ここを手動で更新
# 
# 【考察】
# - スコアが大幅に悪化（-18.38%）
# - マスタテーブル方式が逆効果だった
# - 特徴量が多すぎて過学習の可能性
# - 複雑すぎる特徴量エンジニアリングが予測精度を下げた
# 
# 【今後の改善案】
# - シンプルな特徴量に戻す
# - ハイパーパラメータの調整
# - 段階的な改善アプローチ
# - 過学習対策の実装
# 
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats
import os

In [2]:
base_dir = '../data/'
sales_df = pd.read_csv(base_dir + 'sales_history.csv')
item_categories_df = pd.read_csv(base_dir + 'item_categories.csv')
category_names_df = pd.read_csv(base_dir + 'category_names.csv')
test_df = pd.read_csv(base_dir + 'test.csv')
submission_df = pd.read_csv(base_dir + 'sample_submission.csv', header=None)

In [3]:
# 日付から年と月を抽出
sales_df['年'] = sales_df['日付'].apply(lambda x: x.split('-')[0])
sales_df['月'] = sales_df['日付'].apply(lambda x: x.split('-')[1])

# =============================================================================
# データクレンジング
# =============================================================================

print("データクレンジング前の形状:", sales_df.shape)

# 1. 欠損値の確認と除去
print("\n欠損値の確認:")
print(sales_df.isnull().sum())

# 欠損値がある行を除去
sales_df = sales_df.dropna()
print("欠損値除去後の形状:", sales_df.shape)

# 2. 外れ値の検出と除去（Z-score法）
z_scores = np.abs(stats.zscore(sales_df['売上個数']))
outlier_threshold = 3  # Z-score > 3 を外れ値とする

print(f"\n外れ値の数: {np.sum(z_scores > outlier_threshold)}")
print(f"外れ値の割合: {np.sum(z_scores > outlier_threshold) / len(sales_df) * 100:.2f}%")

# 外れ値を除去
sales_df = sales_df[z_scores <= outlier_threshold]
print("外れ値除去後の形状:", sales_df.shape)

# 3. 売上個数が0以下の異常値を除去
sales_df = sales_df[sales_df['売上個数'] > 0]
print("異常値除去後の形状:", sales_df.shape)

# 月ごとの売上個数を集計
sales_month_df = sales_df.groupby(['商品ID', '店舗ID', '年', '月'])['売上個数'].sum().reset_index()

# 商品カテゴリIDを結合
train_df = pd.merge(sales_month_df, item_categories_df, on='商品ID', how='left')

# データ型を変換
train_df['年'] = train_df['年'].astype(int)
train_df['月'] = train_df['月'].astype(int)

# test_dfにも年と月を追加し、商品カテゴリIDを結合
test_df['年'] = 2022
test_df['月'] = 12
test_df = pd.merge(test_df, item_categories_df, on='商品ID', how='left')

# 特徴量とターゲット変数を定義
feature_columns = ['店舗ID', '商品ID', '年', '月', '商品カテゴリID']
target_column = '売上個数'

# 学習データとテストデータに分割
X_train = train_df[feature_columns]
y_train = train_df[target_column]
X_test = test_df[feature_columns]

データクレンジング前の形状: (1110198, 7)

欠損値の確認:
日付      0
店舗ID    0
商品ID    0
商品価格    0
売上個数    0
年       0
月       0
dtype: int64
欠損値除去後の形状: (1110198, 7)

外れ値の数: 10908
外れ値の割合: 0.98%
外れ値除去後の形状: (1099290, 7)
異常値除去後の形状: (1096692, 7)


In [6]:
# =============================================================================
# マスタテーブル作成（商品マスタ）
# =============================================================================

# 限界利益率を20%と仮定
MARGIN_RATE = 0.20

# データの構造を確認
print("sales_dfのカラム名を確認:")
print(sales_df.columns.tolist())

# 商品マスタの作成
print("\n商品マスタを作成中...")
product_master = sales_df.groupby('商品ID').agg({
    '売上個数': ['mean', 'sum', 'count', 'std']
}).reset_index()

# カラム名を平坦化
product_master.columns = ['商品ID', '平均販売数量', '総販売数量', '販売回数', '販売数量標準偏差']

# 商品カテゴリIDを別途取得（item_categories_dfから）
print("商品カテゴリIDを取得中...")
product_master = product_master.merge(
    item_categories_df[['商品ID', '商品カテゴリID']], 
    on='商品ID', 
    how='left'
)

# 利益重視スコアの計算
product_master['利益重視スコア'] = product_master['平均販売数量'] * MARGIN_RATE
product_master['数量重視スコア'] = product_master['総販売数量'] * MARGIN_RATE

# 商品を4つのカテゴリに分類
product_master['商品利益カテゴリ'] = pd.cut(
    product_master['利益重視スコア'], 
    bins=4, 
    labels=['低利益', '中利益', '高利益', '最高利益']
)

product_master['商品数量カテゴリ'] = pd.cut(
    product_master['数量重視スコア'], 
    bins=4, 
    labels=['低数量', '中数量', '高数量', '最高数量']
)

print("商品マスタの作成完了")
print("商品利益カテゴリの分布:")
print(product_master['商品利益カテゴリ'].value_counts())
print("\n商品数量カテゴリの分布:")
print(product_master['商品数量カテゴリ'].value_counts())

# 商品マスタをCSVファイルとして保存
product_master.to_csv('../data/product_master.csv', index=False, encoding='utf-8')
print(f"\n商品マスタを保存しました: ../data/product_master.csv")
print(f"商品マスタの形状: {product_master.shape}")


sales_dfのカラム名を確認:
['日付', '店舗ID', '商品ID', '商品価格', '売上個数', '年', '月']

商品マスタを作成中...
商品カテゴリIDを取得中...
商品マスタの作成完了
商品利益カテゴリの分布:
商品利益カテゴリ
低利益     8998
中利益       30
高利益        3
最高利益       2
Name: count, dtype: int64

商品数量カテゴリの分布:
商品数量カテゴリ
低数量     8950
中数量       64
高数量       11
最高数量       8
Name: count, dtype: int64

商品マスタを保存しました: ../data/product_master.csv
商品マスタの形状: (9033, 10)


In [7]:
# =============================================================================
# マスタテーブル作成（店舗マスタ）
# =============================================================================

# 店舗マスタの作成
print("店舗マスタを作成中...")
store_master = sales_df.groupby('店舗ID').agg({
    '売上個数': ['mean', 'std', 'sum', 'count'],
    '商品ID': 'nunique'
}).reset_index()

# カラム名を平坦化
store_master.columns = ['店舗ID', '店舗平均販売数量', '店舗販売数量標準偏差', '店舗総販売数量', '店舗販売回数', '店舗商品種類数']

# 店舗を4つのカテゴリに分類
store_master['店舗規模カテゴリ'] = pd.cut(
    store_master['店舗総販売数量'], 
    bins=4, 
    labels=['小規模', '中規模', '大規模', '超大規模']
)

store_master['店舗多様性カテゴリ'] = pd.cut(
    store_master['店舗商品種類数'], 
    bins=4, 
    labels=['低多様性', '中多様性', '高多様性', '最高多様性']
)

print("店舗マスタの作成完了")
print("店舗規模カテゴリの分布:")
print(store_master['店舗規模カテゴリ'].value_counts())
print("\n店舗多様性カテゴリの分布:")
print(store_master['店舗多様性カテゴリ'].value_counts())

# 店舗マスタをCSVファイルとして保存
store_master.to_csv('../data/store_master.csv', index=False, encoding='utf-8')
print(f"\n店舗マスタを保存しました: ../data/store_master.csv")
print(f"店舗マスタの形状: {store_master.shape}")


店舗マスタを作成中...
店舗マスタの作成完了
店舗規模カテゴリの分布:
店舗規模カテゴリ
小規模     10
大規模      3
超大規模     3
中規模      2
Name: count, dtype: int64

店舗多様性カテゴリの分布:
店舗多様性カテゴリ
低多様性     6
中多様性     6
高多様性     4
最高多様性    2
Name: count, dtype: int64

店舗マスタを保存しました: ../data/store_master.csv
店舗マスタの形状: (18, 8)


In [9]:
# =============================================================================
# マスタテーブルを使用した特徴量統合
# =============================================================================

# 月ごとの売上個数を集計
sales_month_df = sales_df.groupby(['商品ID', '店舗ID', '年', '月'])['売上個数'].sum().reset_index()

# 学習データの作成（商品マスタと店舗マスタを結合）
print("学習データを作成中...")
train_df = sales_month_df.copy()

# 商品マスタを結合
train_df = train_df.merge(product_master, on='商品ID', how='left')

# 店舗マスタを結合
train_df = train_df.merge(store_master, on='店舗ID', how='left')

# データ型を変換
train_df['年'] = train_df['年'].astype(int)
train_df['月'] = train_df['月'].astype(int)

print(f"学習データの形状: {train_df.shape}")
print("学習データのカラム名:")
print(train_df.columns.tolist())

# テストデータの準備
print("\nテストデータを準備中...")
test_df['年'] = 2022
test_df['月'] = 12

# テストデータに商品マスタを結合
test_df = test_df.merge(product_master, on='商品ID', how='left')

# テストデータに店舗マスタを結合
test_df = test_df.merge(store_master, on='店舗ID', how='left')

print(f"テストデータの形状: {test_df.shape}")
print("テストデータのカラム名:")
print(test_df.columns.tolist())

# 重複カラムの処理
print("\n重複カラムを処理中...")
if '商品カテゴリID_x' in test_df.columns and '商品カテゴリID_y' in test_df.columns:
    # 商品カテゴリIDの重複を解決
    test_df['商品カテゴリID'] = test_df['商品カテゴリID_x'].fillna(test_df['商品カテゴリID_y'])
    test_df = test_df.drop(['商品カテゴリID_x', '商品カテゴリID_y'], axis=1)
    print("✅ 商品カテゴリIDの重複を解決しました")

# 不要なカラムを削除
if 'index' in test_df.columns:
    test_df = test_df.drop('index', axis=1)
    print("✅ 'index'カラムを削除しました")

print(f"処理後のテストデータの形状: {test_df.shape}")
print("処理後のテストデータのカラム名:")
print(test_df.columns.tolist())

# 欠損値の処理（新商品や新店舗の場合）
print("\n欠損値を処理中...")

# カテゴリカル変数の欠損値を埋める
categorical_cols = ['商品利益カテゴリ', '商品数量カテゴリ', '店舗規模カテゴリ', '店舗多様性カテゴリ']
for col in categorical_cols:
    if col in test_df.columns:
        default_value = '中' + col.replace('カテゴリ', '')
        test_df[col] = test_df[col].fillna(default_value)
        print(f"✅ {col}の欠損値を'{default_value}'で埋めました")

# 数値変数の欠損値を埋める
numeric_cols = ['平均販売数量', '総販売数量', '店舗平均販売数量', '店舗総販売数量']
for col in numeric_cols:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(test_df[col].mean())
        print(f"✅ {col}の欠損値を平均値で埋めました")

# 特徴量とターゲット変数を定義
feature_columns = [
    '店舗ID', '商品ID', '年', '月', '商品カテゴリID',
    '商品利益カテゴリ', '商品数量カテゴリ', '店舗規模カテゴリ', '店舗多様性カテゴリ',
    '平均販売数量', '総販売数量', '店舗平均販売数量', '店舗総販売数量'
]
target_column = '売上個数'

# 特徴量の存在確認
print("\n特徴量の存在確認中...")
missing_train_cols = [col for col in feature_columns if col not in train_df.columns]
missing_test_cols = [col for col in feature_columns if col not in test_df.columns]

if missing_train_cols:
    print(f"⚠️ 学習データに不足しているカラム: {missing_train_cols}")
if missing_test_cols:
    print(f"⚠️ テストデータに不足しているカラム: {missing_test_cols}")

# 存在する特徴量のみを使用
available_features = [col for col in feature_columns if col in train_df.columns and col in test_df.columns]
print(f"✅ 使用可能な特徴量数: {len(available_features)}")
print(f"使用可能な特徴量: {available_features}")

# 学習データとテストデータに分割
X_train = train_df[available_features]
y_train = train_df[target_column]
X_test = test_df[available_features]

print(f"\n最終的な特徴量数: {len(feature_columns)}")
print("特徴量一覧:", feature_columns)
print(f"学習データ形状: {X_train.shape}")
print(f"テストデータ形状: {X_test.shape}")

# データ型を確認
print("\n学習データのデータ型:")
print(X_train.dtypes)
print("\nテストデータのデータ型:")
print(X_test.dtypes)


学習データを作成中...
学習データの形状: (492614, 21)
学習データのカラム名:
['商品ID', '店舗ID', '年', '月', '売上個数', '平均販売数量', '総販売数量', '販売回数', '販売数量標準偏差', '商品カテゴリID', '利益重視スコア', '数量重視スコア', '商品利益カテゴリ', '商品数量カテゴリ', '店舗平均販売数量', '店舗販売数量標準偏差', '店舗総販売数量', '店舗販売回数', '店舗商品種類数', '店舗規模カテゴリ', '店舗多様性カテゴリ']

テストデータを準備中...
テストデータの形状: (3060, 38)
テストデータのカラム名:
['index', '商品ID', '店舗ID', '年', '月', '商品カテゴリID_x', '平均販売数量_x', '総販売数量_x', '販売回数_x', '販売数量標準偏差_x', '商品カテゴリID_y', '利益重視スコア_x', '数量重視スコア_x', '商品利益カテゴリ_x', '商品数量カテゴリ_x', '店舗平均販売数量_x', '店舗販売数量標準偏差_x', '店舗総販売数量_x', '店舗販売回数_x', '店舗商品種類数_x', '店舗規模カテゴリ_x', '店舗多様性カテゴリ_x', '平均販売数量_y', '総販売数量_y', '販売回数_y', '販売数量標準偏差_y', '商品カテゴリID', '利益重視スコア_y', '数量重視スコア_y', '商品利益カテゴリ_y', '商品数量カテゴリ_y', '店舗平均販売数量_y', '店舗販売数量標準偏差_y', '店舗総販売数量_y', '店舗販売回数_y', '店舗商品種類数_y', '店舗規模カテゴリ_y', '店舗多様性カテゴリ_y']

重複カラムを処理中...
✅ 商品カテゴリIDの重複を解決しました
✅ 'index'カラムを削除しました
処理後のテストデータの形状: (3060, 35)
処理後のテストデータのカラム名:
['商品ID', '店舗ID', '年', '月', '平均販売数量_x', '総販売数量_x', '販売回数_x', '販売数量標準偏差_x', '利益重視スコア_x', '数量重視スコア_x', '商品利益カテゴリ_x', '商品

In [10]:
# =============================================================================
# モデルの学習（n_estimators=100に戻してベースラインに設定）
# =============================================================================

# モデルの定義と学習（RandomForestRegressorを使用）
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 予測結果に負の値があれば0に変換
y_pred[y_pred < 0] = 0

print("モデルの学習が完了しました")
print(f"予測結果の形状: {y_pred.shape}")
print(f"予測値の範囲: {y_pred.min():.2f} ～ {y_pred.max():.2f}")

# 特徴量重要度の表示
feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n特徴量重要度（上位10位）:")
print(feature_importance.head(10))

モデルの学習が完了しました
予測結果の形状: (3060,)
予測値の範囲: 1.00 ～ 20.76

特徴量重要度（上位10位）:
    feature  importance
1      商品ID    0.566143
0      店舗ID    0.235302
3         月    0.168293
2         年    0.029014
4  商品カテゴリID    0.001248


In [11]:
# =============================================================================
# 提出ファイルの作成
# =============================================================================

# submission_df の 1 列に予測結果 (y_pred) を代入
submission_df[1] = y_pred

# 提出ファイルの保存
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%m%d-%H%M")  # MMdd-HHmm形式
improvement_name = "master"
submission_file_name = f'../submissions/20250909_Exercises3_Challenge_{improvement_name}_{timestamp}.csv'
submission_df.to_csv(submission_file_name, index=False, header=False)
print(f"提出ファイルを保存しました: {submission_file_name}")

# 現在の日時を取得してタイムスタンプ付きファイル名を作成
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%m%d-%H%M")  # MMdd-HHmm形式

# 提出用ファイル名（改善内容とタイムスタンプ付き）
# 注意: 新しい改善を試す際は、以下の改善内容部分を変更してください
improvement_name = "rf100"  # 例: "rf200", "xgb", "features", "ensemble" など
submission_file_name = f'../submissions/20250909_Exercises3_Challenge_{improvement_name}_{timestamp}.csv'

# 提出用データフレームをCSVファイルとして出力
submission_df.to_csv(submission_file_name, index=False, header=False)

print(f"\n提出用ファイル '{submission_file_name}' が作成されました。")
print("このファイルをコンペティションサイトに提出してください。")

提出ファイルを保存しました: ../submissions/20250909_Exercises3_Challenge_master_0909-1430.csv

提出用ファイル '../submissions/20250909_Exercises3_Challenge_rf100_0909-1430.csv' が作成されました。
このファイルをコンペティションサイトに提出してください。


In [ ]:
# =============================================================================
# スコア記録（提出後に値を入力）
# =============================================================================

from score_analysis import record_score, compare_scores, generate_improvement_summary

# 提出後にスコアを入力してください
current_score = None  # 提出後に実際のスコアを入力
model_name = "RandomForestRegressor (n_estimators=100) + マスタテーブル方式"
features_used = "店舗ID, 商品ID, 年, 月, 商品カテゴリID, 商品利益カテゴリ, 商品数量カテゴリ, 店舗規模カテゴリ, 店舗多様性カテゴリ, 平均販売数量, 総販売数量, 店舗平均販売数量, 店舗総販売数量"
notes = "データクレンジング + マスタテーブル方式 + 利益重視特徴量 + 店舗補正特徴量"

if current_score is not None:
    # スコアを記録・分析
    result = record_score(
        current_score=current_score,
        model_name=model_name,
        features_used=features_used,
        notes=notes
    )
else:
    print("提出後にスコアを記録してください")
    print("例: current_score = 3.500000")
    print("スコアを入力後、このセルを再実行してください")


In [13]:
# =============================================================================
# スコア記録・分析セル
# =============================================================================
# 提出後にスコアを記録してください

# スコア記録・分析ライブラリをインポート
from score_analysis import record_score, compare_scores, generate_improvement_summary

# スコア記録（提出後に値を入力）
current_score = 3.644648498790873  # 提出後にスコアを入力
model_name = "RandomForestRegressor (n_estimators=100)"
features_used = "店舗ID, 商品ID, 年, 月, 商品カテゴリID"
notes = "線形回帰からRandomForestに変更"

if current_score is not None:
    # スコアを記録・分析
    result = record_score(
        current_score=current_score,
        model_name=model_name,
        features_used=features_used,
        notes=notes
    )
else:
    print("スコアを記録してください。")
    print("例: current_score = 3.500000")
    print("その後、このセルを再実行してください。")


📊 スコア分析結果
前回のスコア: 3.078812
今回のスコア: 3.644648
改善幅: -0.565837
改善率: -18.38%
ベースラインからの改善率: 8.74%
使用モデル: RandomForestRegressor (n_estimators=100)
使用特徴量: 店舗ID, 商品ID, 年, 月, 商品カテゴリID
メモ: 線形回帰からRandomForestに変更
------------------------------------------------------------
❌ スコアが悪化しました。改善が必要です。
